# COURSERA Applied Data Science Capstone

## Week 3: Segmenting and Clustering Neighborhoods in Toronto

### Part 1

Import all the required libraries

In [1]:
#!conda install -c conda-forge beautifulsoup4 --yes # Only needed one time
import pandas as pd
import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M , in order to obtain the data that is in the table of postal codes  
Parse using BeautifulSoup

In [2]:
wikiPage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
rawWikiPage = requests.get(wikiPage).text
parseWiki = BeautifulSoup(rawWikiPage, 'html.parser')

print('Wiki page scraped and parsed')

Wiki page scraped and parsed


Extract the table and put into Pandas dataframe  

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
postalCodesTable = []
for tr in parseWiki.tbody.find_all('tr'):
    postalCodesTable.append([ td.get_text().strip() for td in tr.find_all('td')])
rawWikiDF = pd.DataFrame(postalCodesTable, columns=['PostalCode', 'Borough', 'Neighborhood'])
rawWikiDF

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
...,...,...,...
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor


Keep only the cells that have an assigned borough. Drop records that are all "None" and ignore cells with a borough that is "Not assigned".

In [4]:
workingWikiDF = rawWikiDF.dropna()
workingWikiDF = workingWikiDF[workingWikiDF.Borough != "Not assigned"].reset_index(drop=True)
workingWikiDF

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West
207,M8Z,Etobicoke,Mimico NW
208,M8Z,Etobicoke,The Queensway West
209,M8Z,Etobicoke,Royal York South West


Combine rows of neighborhoods that have the same postal code area.

In [5]:
workingWikiDF_groupNeighborhoods = workingWikiDF.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
workingWikiDF_groupNeighborhoods

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Checking the value of postal code M7A for next step

In [6]:
workingWikiDF_groupNeighborhoods[workingWikiDF_groupNeighborhoods['PostalCode']=='M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


Assign the borough name as the neighborhood for all neighborhoods with "Not assigned" as neighborhood name.

In [7]:
for index, row in workingWikiDF_groupNeighborhoods.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
wikiDF = workingWikiDF_groupNeighborhoods

wikiDF

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Checking the value of postal code M7A to ensure last step worked

In [8]:
wikiDF[wikiDF['PostalCode']=='M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [9]:
wikiDF.shape

(103, 3)